In [1]:
import matplotlib.pyplot as plt
import os

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Dataset Construction

#### Load dataset

In [5]:
background_dataset = pd.read_csv('../../source-interdiction/training_testing_data/background_template_dataset.csv')
source_dataset = pd.read_csv('../../source-interdiction/training_testing_data/shielded_templates_200kev_dataset.csv')

#### Dataset details

In [6]:
print('sourcedist: ' + str(sorted(set(source_dataset['sourcedist']))))
print('sourceheight: ' + str(sorted(set(source_dataset['sourceheight']))))
print('alum shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='alum']['shieldingdensity']))))
print('iron shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='iron']['shieldingdensity']))))
print('lead shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='lead']['shieldingdensity']))))
print('fwhm: ' + str(sorted(set(source_dataset['fwhm']))))

sourcedist: [50.0, 112.5, 175.0, 237.5, 300.0]
sourceheight: [50.0, 75.0, 100.0, 125.0, 150.0]
alum shieldingdensity: [1.82, 4.18, 7.49, 13.16]
iron shieldingdensity: [1.53, 3.5, 6.28, 11.02]
lead shieldingdensity: [0.22, 0.51, 0.92, 1.61]
fwhm: [6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]


#### Customize dataset

In [7]:
source_dataset = source_dataset[(source_dataset['fwhm']==7.0) | 
                                (source_dataset['fwhm']==7.5) |
                                (source_dataset['fwhm']==8.0)]

source_dataset = source_dataset[(source_dataset['sourcedist']==50.5) | 
                                (source_dataset['sourcedist']==175.0) | 
                                (source_dataset['sourcedist']==300.0)]

source_dataset = source_dataset[(source_dataset['sourceheight']==50.0) |
                                (source_dataset['sourceheight']==100.0) |
                                (source_dataset['sourceheight']==150.0)]

# remove 80% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=13.16]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=11.02]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.61]

#### Remove empty spectra

In [8]:
zero_count_indicies = np.argwhere(np.sum(source_dataset.values[:,6:],axis=1) == 0).flatten()

print('indicies dropped: ' +str(zero_count_indicies))

source_dataset.drop(source_dataset.index[zero_count_indicies], inplace=True)

indicies dropped: [552 553 554 555 556 557 564 565 566 582 583 584 585 586 587 594 595 596
 612 613 614 615 616 617 624 625 626 642 643 644 645 646 647 654 655 656
 672 673 674 675 676 677 684 685 686 702 703 704 705 706 707 714 715 716]


#### Add empty spectra for background 

In [9]:
blank_spectra = []
for fwhm in set(source_dataset['fwhm']):
    num_examples = source_dataset[(source_dataset['fwhm']==fwhm) &
                                  (source_dataset['isotope']==source_dataset['isotope'].iloc()[0])].shape[0]
    for k in range(num_examples):
        blank_spectra_tmp = [0]*1200
        blank_spectra_tmp[5] = fwhm
        blank_spectra_tmp[0] = 'background'
        blank_spectra_tmp[3] = 'background'
        blank_spectra.append(blank_spectra_tmp)

source_dataset = source_dataset.append(pd.DataFrame(blank_spectra,
                                                    columns=source_dataset.columns))

#### Create dataset from spectra

In [10]:
spectra_dataset = source_dataset.values[:,5:].astype('float64')
all_keys = source_dataset['isotope'].values

## Define Training Parameters

#### Define online data augmentation

In [11]:
def integration_time():
    return np.random.uniform(10,600)

def background_cps():
    return np.random.poisson(200)

def signal_to_background():
    return np.random.uniform(0.5,2)

def calibration():
    return [np.random.uniform(0,20),
            np.random.uniform(2500/3000,3500/3000),
            0]

online_data_augmentation = online_data_augmentation_vanilla(background_dataset,
                                background_cps,
                                integration_time,
                                signal_to_background,
                                calibration,)

## Load testing dataset 

In [12]:
testing_dataset = np.load('../dataset_generation/testing_dataset_full_200keV_1000.npy')

In [13]:
testing_spectra = np.add(testing_dataset.item()['sources'], testing_dataset.item()['backgrounds'])
testing_keys = testing_dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))
testing_keys_binarized = mlb.transform(testing_keys)
training_keys_binarized = mlb.transform(all_keys)

# Train network

### Define hyperparameters

In [14]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='DNN-onlinedataaugfull'

In [16]:
def make_model():
    '''
    Makes a random model given some parameters.

    '''
    number_layers = choice([1, 2, 3])
    dense_nodes = 2**np.random.randint(5, 10, number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    model_features = dnn_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-2,0),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=training_keys_binarized.shape[1],
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.tanh,tf.nn.relu,tf.nn.sigmoid]),
        scaler=choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                       make_pipeline(FunctionTransformer(np.sqrt, validate=True))]))

    model = DNN(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
testing_errors = []

for network_id in range(number_hyperparameters_to_search):
    # reset model on each iteration
    model, model_features = make_model()
    
    model_features.activation_function = tf.nn.sigmoid
    model_features.batch_size = 32
    model_features.dense_nodes = [256]
    model_features.dropout_probability = 0.63
    model_features.l2_regularization_scale = 0.04293
    model_features.learining_rate = 0.0001212
    model_features.scaler = make_pipeline(FunctionTransformer(np.sqrt, validate=True))
    
    
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    
    costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
            training_data=spectra_dataset,
            training_keys=training_keys_binarized,
            testing_data=testing_spectra,
            testing_keys=testing_keys_binarized,
            model=model,
            optimizer=optimizer,
            num_epochs=500,
            obj_cost=model.cross_entropy,
            earlystop_cost_fn=model.f1_error,
            earlystop_patience=10,
            verbose=True,
            fit_batch_verbose=1,
            data_augmentation=online_data_augmentation)
    testing_errors.append(earlystop_errors_tmp)

    # np.save('./final-models/final_test_errors_'+model_id, training_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

Epoch 1: CostFunc loss: 0.00 11.14, EarlyStop loss: 0.00 0.81
Epoch 2: CostFunc loss: 0.00 9.12, EarlyStop loss: 0.00 0.72
Epoch 3: CostFunc loss: 0.00 7.64, EarlyStop loss: 0.00 0.67
Epoch 4: CostFunc loss: 0.00 6.52, EarlyStop loss: 0.00 0.65
Epoch 5: CostFunc loss: 0.00 5.68, EarlyStop loss: 0.00 0.62
Epoch 6: CostFunc loss: 0.00 4.99, EarlyStop loss: 0.00 0.53
Epoch 7: CostFunc loss: 0.00 4.51, EarlyStop loss: 0.00 0.58
Epoch 8: CostFunc loss: 0.00 4.10, EarlyStop loss: 0.00 0.51
Epoch 9: CostFunc loss: 0.00 3.77, EarlyStop loss: 0.00 0.51
Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.46
Epoch 11: CostFunc loss: 0.00 3.26, EarlyStop loss: 0.00 0.47
Epoch 12: CostFunc loss: 0.00 3.05, EarlyStop loss: 0.00 0.44
Epoch 13: CostFunc loss: 0.00 2.93, EarlyStop loss: 0.00 0.45
Epoch 14: CostFunc loss: 0.00 2.79, EarlyStop loss: 0.00 0.41
Epoch 15: CostFunc loss: 0.00 2.67, EarlyStop loss: 0.00 0.40
Epoch 16: CostFunc loss: 0.00 2.58, EarlyStop loss: 0.00 0.40
Epoch 17: CostFu